ANALYSIS

In [30]:
import sqlite3
import pandas as pd
import numpy as np

In [31]:
import os 
os.chdir('/Users/ayokunle/Documents/mylab/codecademy_data_pipeline_proj/src')


In [32]:
# Create connection object
db = sqlite3.connect('cademycode.db')
# Create cursor object
cursor = db.cursor()

In [33]:
# get table names
tables = [table for table in cursor.execute("SELECT name FROM sqlite_schema WHERE type='table' ORDER BY name;")]
print(tables)

[('cademycode_courses',), ('cademycode_student_jobs',), ('cademycode_students',)]


In [34]:
courses = pd.read_sql('SELECT * FROM cademycode_courses', db)
student_jobs = pd.read_sql_query('SELECT * FROM cademycode_student_jobs', db)
students = pd.read_sql_query('SELECT * FROM cademycode_students', db)
# courses.head()

In [35]:
print(f'Courses dataframe size:{courses.shape}')
print(f'Students Jobs dataframe size:{student_jobs.shape}')
print(f'Students dataframe size: {students.shape}')

Courses dataframe size:(10, 3)
Students Jobs dataframe size:(13, 3)
Students dataframe size: (5000, 9)


In [36]:
dfs = [courses, student_jobs, students]
for df in dfs:
    print("="*50)
    print(df.isna().sum())
    print("\n")
    print(df.dtypes)
    print("\n")

career_path_id       0
career_path_name     0
hours_to_complete    0
dtype: int64


career_path_id        int64
career_path_name     object
hours_to_complete     int64
dtype: object


job_id          0
job_category    0
avg_salary      0
dtype: int64


job_id           int64
job_category    object
avg_salary       int64
dtype: object


uuid                        0
name                        0
dob                         0
sex                         0
contact_info                0
job_id                      5
num_course_taken          251
current_career_path_id    471
time_spent_hrs            471
dtype: int64


uuid                       int64
name                      object
dob                       object
sex                       object
contact_info              object
job_id                    object
num_course_taken          object
current_career_path_id    object
time_spent_hrs            object
dtype: object




In [37]:
students.head()

,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
0,1,Annabelle Avery,1943-07-03,F,"{""mailing_address"": ""303 N Timber Key, Irondal...",7.0,6.0,1.0,4.99
1,2,Micah Rubio,1991-02-07,M,"{""mailing_address"": ""767 Crescent Fair, Shoals...",7.0,5.0,8.0,4.4
2,3,Hosea Dale,1989-12-07,M,"{""mailing_address"": ""P.O. Box 41269, St. Bonav...",7.0,8.0,8.0,6.74
3,4,Mariann Kirk,1988-07-31,F,"{""mailing_address"": ""517 SE Wintergreen Isle, ...",6.0,7.0,9.0,12.31
4,5,Lucio Alexander,1963-08-31,M,"{""mailing_address"": ""18 Cinder Cliff, Doyles b...",7.0,14.0,3.0,5.64


In [38]:
# exploding contact info field 
import json
students['contact_info'] = students['contact_info'].map(lambda x: json.loads(x) if isinstance(x, str) else x )

# explode 
df_contact_info = pd.json_normalize(students['contact_info'])

# join
students = pd.concat([students, df_contact_info], axis='columns')
students = students.drop(['contact_info'], axis='columns')

In [39]:
students.head()

,uuid,name,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,mailing_address,email
0,1,Annabelle Avery,1943-07-03,F,7.0,6.0,1.0,4.99,"303 N Timber Key, Irondale, Wisconsin, 84736",annabelle_avery9376@woohoo.com
1,2,Micah Rubio,1991-02-07,M,7.0,5.0,8.0,4.4,"767 Crescent Fair, Shoals, Indiana, 37439",rubio6772@hmail.com
2,3,Hosea Dale,1989-12-07,M,7.0,8.0,8.0,6.74,"P.O. Box 41269, St. Bonaventure, Virginia, 83637",hosea_dale8084@coldmail.com
3,4,Mariann Kirk,1988-07-31,F,6.0,7.0,9.0,12.31,"517 SE Wintergreen Isle, Lane, Arkansas, 82242",kirk4005@hmail.com
4,5,Lucio Alexander,1963-08-31,M,7.0,14.0,3.0,5.64,"18 Cinder Cliff, Doyles borough, Rhode Island,...",alexander9810@hmail.com


In [40]:
students.isna().any()

uuid                      False
name                      False
dob                       False
sex                       False
job_id                     True
num_course_taken           True
current_career_path_id     True
time_spent_hrs             True
mailing_address           False
email                     False
dtype: bool

DEALING WITH MISSING DATA

In [41]:
# missing_num_course = students[students[['num_course_taken']].isnull().any(axis='columns')]
# missing_time_spent = students[students[['time_spent_hrs']].isnull().any(axis='columns')]
# missing_job_id = students[students[['job_id']].isnull().any(axis='columns')]

In [42]:
# missing_num_course.info()

In [43]:
# missing_num_course.head()

In [44]:
# missing_time_spent.head()

In [45]:
# missing_time_spent.isna().sum()

In [46]:
# missing_job_id.head()

In [47]:
# missing_job_id.info()

In [48]:
# student_jobs

In [49]:
# rows_with_missing_data = students[students.isnull().any(axis='columns')]

In [50]:
# incomplete_data = rows_with_missing_data[~( 
#     (rows_with_missing_data.num_course_taken.isna()) &
#     (rows_with_missing_data.current_career_path_id.isna()) &
#     (rows_with_missing_data.time_spent_hrs).isna()
# )]

In [51]:
# incomplete_data.isna().sum()

In [52]:
# # assume that students that have no career path, num_course_taken, and time_spent_hrs are registered 
# # but have not started learning
# rows_with_missing_data = students[students.isnull().any(axis='columns')]

# # every thing other than that is incomplete and will be stored separately
# incomplete_data = rows_with_missing_data[~( 
#     (rows_with_missing_data.num_course_taken.isna()) &
#     (rows_with_missing_data.current_career_path_id.isna()) &
#     (rows_with_missing_data.time_spent_hrs).isna()
# )]

In [53]:
# incomplete_data.isna().sum()

In [54]:
# assuming students with no jobs have a no_job category
# students['job_id'].unique()

# I create  a new catergory in student jobs and fill the students dataframe with that id to maintain integrity: job_id 999, 
new_job_category = {'job_id': 999, 'job_category': 'undefined', 'avg_salary': 0}
student_jobs.loc[len(student_jobs)] = new_job_category
students['job_id'] = students['job_id'].fillna(999)


In [55]:
# all students without a career path have also spent zero hours learning 
# create a new category called (no_category) and assign it an id in the courses table
# fill the table with 0 for time spent and 999 for career path id
new_career_path = {'career_path_id': 999, 'career_path_name':	'undefined', 'hours_to_complete': 0}
courses.loc[len(courses)] = new_career_path

# students that have not started learning or signed up for any career path
# i.e num_course_taken, current_career_path_id and time_spent_hrs is null 

conditions = ( 
    (students.num_course_taken.isna()) &
    (students.current_career_path_id.isna()) &
    (students.time_spent_hrs).isna()
)
students.loc[conditions, :] = students.loc[conditions, :].fillna({'num_course_taken': 0, 'current_career_path_id': 999,'time_spent_hrs': 0})

# students.loc[( 
#     (students.num_course_taken.isna()) &
#     (students.current_career_path_id.isna()) &
#     (students.time_spent_hrs).isna()
# )].fillna({'num_course_taken': 0, 'current_career_path_id': '999','time_spent_hrs': 0}, inplace= True)


# structurally missing data where students havent selected career path so time spent is 0 
students = students.fillna({'current_career_path_id': 999,'time_spent_hrs': 0})


In [56]:
# conditions = ( 
#     (students.num_course_taken.isna()) &
#     (students.current_career_path_id.isna()) &
#     (students.time_spent_hrs).isna()
# )
# students.loc[conditions, :] = students.loc[conditions, :].fillna({'num_course_taken': 0, 'current_career_path_id': '999','time_spent_hrs': 0})

In [57]:
# students.loc[conditions, :]

In [58]:
students.isna().sum()

uuid                        0
name                        0
dob                         0
sex                         0
job_id                      0
num_course_taken          231
current_career_path_id      0
time_spent_hrs              0
mailing_address             0
email                       0
dtype: int64

In [59]:
incomplete_data = students[students.isna().any(axis=1)]

In [60]:
students = students.dropna()

In [61]:
students.dtypes

uuid                       int64
name                      object
dob                       object
sex                       object
job_id                    object
num_course_taken          object
current_career_path_id    object
time_spent_hrs            object
mailing_address           object
email                     object
dtype: object

In [62]:
# convert dob to datetime, 
# num_course_taken, current_career_path_id,job_id to integer, 
# time_spent_hrs to float

students['dob'] = students['dob'].astype('datetime64[ns]')

students['num_course_taken'] = students['num_course_taken'].astype(float)
students['current_career_path_id'] = students['current_career_path_id'].astype(float).astype(int)
students['job_id'] = students['job_id'].astype(float).astype(int)

students['time_spent_hrs'] = students['time_spent_hrs'].astype(float)


In [63]:
students.dtypes

uuid                               int64
name                              object
dob                       datetime64[ns]
sex                               object
job_id                             int64
num_course_taken                 float64
current_career_path_id             int64
time_spent_hrs                   float64
mailing_address                   object
email                             object
dtype: object

STUDENT JOBS TABLE

In [64]:
student_jobs

,job_id,job_category,avg_salary
0,1,analytics,86000
1,2,engineer,101000
2,3,software developer,110000
3,4,creative,66000
4,5,financial services,135000
5,6,education,61000
6,7,HR,80000
7,8,student,10000
8,9,healthcare,120000
9,0,other,80000


In [65]:
# drop duplicates id 3,4,5
student_jobs = student_jobs.drop_duplicates().reset_index(drop=True)

In [66]:
student_jobs

,job_id,job_category,avg_salary
0,1,analytics,86000
1,2,engineer,101000
2,3,software developer,110000
3,4,creative,66000
4,5,financial services,135000
5,6,education,61000
6,7,HR,80000
7,8,student,10000
8,9,healthcare,120000
9,0,other,80000


CAREER PATHS (COURSES TABLE)

In [67]:
courses.dtypes

career_path_id        int64
career_path_name     object
hours_to_complete     int64
dtype: object

In [68]:
courses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   career_path_id     11 non-null     int64 
 1   career_path_name   11 non-null     object
 2   hours_to_complete  11 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 352.0+ bytes


In [69]:
courses

,career_path_id,career_path_name,hours_to_complete
0,1,data scientist,20
1,2,data engineer,20
2,3,data analyst,12
3,4,software engineering,25
4,5,backend engineer,18
5,6,frontend engineer,20
6,7,iOS developer,27
7,8,android developer,27
8,9,machine learning engineer,35
9,10,ux/ui designer,15


In [70]:
# sanity check
print(students.info())
print(student_jobs.info())
print(courses.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4769 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    4769 non-null   int64         
 1   name                    4769 non-null   object        
 2   dob                     4769 non-null   datetime64[ns]
 3   sex                     4769 non-null   object        
 4   job_id                  4769 non-null   int64         
 5   num_course_taken        4769 non-null   float64       
 6   current_career_path_id  4769 non-null   int64         
 7   time_spent_hrs          4769 non-null   float64       
 8   mailing_address         4769 non-null   object        
 9   email                   4769 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 409.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (t

In [71]:
# create analytics-ready SQLite database and flat CSV file
# join all tables to a single denormalized table
analytics_df = pd.merge(students, student_jobs, how='inner', on='job_id')
analytics_df = pd.merge(analytics_df, courses, how='inner', left_on='current_career_path_id', right_on='career_path_id')

analytics_df.to_csv("analytics.csv")

In [ ]:
# to table
analytics_df.to_sql('analytics', if_exists='append', con=db)